# Settings

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

# Loading the data

In [2]:
sample_submission = pd.read_csv("C:/Users/odeli/Desktop/dataCamp_data/sample_submission.csv")
test = pd.read_csv("C:/Users/odeli/Desktop/dataCamp_data/test.csv")
specs = pd.read_csv("C:/Users/odeli/Desktop/dataCamp_data/specs.csv")
train_labels = pd.read_csv("C:/Users/odeli/Desktop/dataCamp_data/train_labels.csv")
train = pd.read_csv("C:/Users/odeli/Desktop/dataCamp_data/train.csv")

In [3]:
print('Shapes:\n - train: ', np.shape(train), '\n - train_labels: ',np.shape(train_labels), '\n - test: ',np.shape(test), '\n - specs: ',np.shape(specs), '\n - sample_submission: ',np.shape(sample_submission))

Shapes:
 - train:  (11341042, 11) 
 - train_labels:  (17690, 7) 
 - test:  (1156414, 11) 
 - specs:  (386, 3) 
 - sample_submission:  (1000, 2)


## Making labels

The first very important thing to do is to create the labels.
As we said the right information is contained in the event_data column.
This column is a dictionary with a variable size for each line.

Organisation of the information recorded by the application:
    * When a child downloads the app an id is created: id_installation
    * When he starts playing a game an other id is created: game_session
    * Inside a game_session many things can be done by the gamer. Every event has its own id: event_id. The different possible events are caracterized by a code contained in the event_code column.
The only interesting events to build the labels are the lasts one in a game session: they correspond to the succes or the failure of the game in a given game_session. The corrresponding event_code is 4100 for 4 of the 5 games and 4110 for the last one.

In [4]:
train[train['title'].str.contains('(Assessment)')].groupby('title').count()

C:\Users\odeli\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,type,world
title,,,,,,,,,,
Bird Measurer (Assessment),190164,190164,190164,190164,190164,190164,190164,190164,190164,190164
Cart Balancer (Assessment),163343,163343,163343,163343,163343,163343,163343,163343,163343,163343
Cauldron Filler (Assessment),181925,181925,181925,181925,181925,181925,181925,181925,181925,181925
Chest Sorter (Assessment),190318,190318,190318,190318,190318,190318,190318,190318,190318,190318
Mushroom Sorter (Assessment),199595,199595,199595,199595,199595,199595,199595,199595,199595,199595


In [19]:
train[train['title'].str.contains('Bird Measurer')].groupby('event_code').count()

,event_id,game_session,timestamp,event_data,installation_id,event_count,game_time,title,type,world
event_code,,,,,,,,,,
2000,3113,3113,3113,3113,3113,3113,3113,3113,3113,3113
2010,1430,1430,1430,1430,1430,1430,1430,1430,1430,1430
2020,4951,4951,4951,4951,4951,4951,4951,4951,4951,4951
2030,3275,3275,3275,3275,3275,3275,3275,3275,3275,3275
3010,10498,10498,10498,10498,10498,10498,10498,10498,10498,10498
3020,10009,10009,10009,10009,10009,10009,10009,10009,10009,10009
3021,1860,1860,1860,1860,1860,1860,1860,1860,1860,1860
3110,10269,10269,10269,10269,10269,10269,10269,10269,10269,10269
3120,9819,9819,9819,9819,9819,9819,9819,9819,9819,9819


In [22]:
tmp = train[train['title'].str.contains('Bird Measurer')]
np.shape(tmp)

(190164, 11)

In [23]:
tmp = tmp[tmp['event_code'] == 4110]
np.shape(tmp)

(11158, 11)

In [55]:
def get_accuracy(data, game, code):
    
    # Filtre par 1 des 5 jeux (game)
    tmp = data[data['title'].str.contains(game)]
    
    # Filtre dernier event = assessment 4110/4100 (code)
    tmp = tmp[tmp['event_code'] == code]
    
    # Colonne status
    status = ["NA" for i in range(np.shape(tmp)[0])]
    for i in range(np.shape(tmp)[0]):
        if ('correct":false' in tmp.loc[tmp.index[i], 'event_data']):
            status[i] = 'incorrect'
        elif ('correct":true' in tmp.loc[tmp.index[i], 'event_data']):
            status[i] = 'correct'
        else:
            status[i] = 'NA'
    tmp['status'] = status
            
    return(tmp)
    

In [57]:
BM_labels = get_accuracy(train, 'Bird Measurer', 4110)

In [59]:
BM_labels.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,status
2709,17113b36,77b8ee947eb84b4e,2019-08-06T05:35:54.898Z,"{""correct"":false,""caterpillars"":[11,8,3],""even...",0006a69f,29,4110,35771,Bird Measurer (Assessment),Assessment,TREETOPCITY,incorrect
2715,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:01.927Z,"{""correct"":false,""caterpillars"":[11,8,11],""eve...",0006a69f,35,4110,42805,Bird Measurer (Assessment),Assessment,TREETOPCITY,incorrect
2720,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:06.512Z,"{""correct"":false,""caterpillars"":[11,8,5],""even...",0006a69f,40,4110,47388,Bird Measurer (Assessment),Assessment,TREETOPCITY,incorrect
2725,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:09.739Z,"{""correct"":false,""caterpillars"":[11,8,7],""even...",0006a69f,45,4110,50605,Bird Measurer (Assessment),Assessment,TREETOPCITY,incorrect
2730,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:13.951Z,"{""correct"":false,""caterpillars"":[11,8,4],""even...",0006a69f,50,4110,54822,Bird Measurer (Assessment),Assessment,TREETOPCITY,incorrect


In [62]:
BM_labels.groupby(('installation_id','game_session','title')).count()

C:\Users\odeli\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


event_id  \
installation_id game_session     title                                  
0006a69f        77b8ee947eb84b4e Bird Measurer (Assessment)        11   
                a9ef3ecb3d1acc6a Bird Measurer (Assessment)         1   
0006c192        957406a905d59afd Bird Measurer (Assessment)         2   
00129856        ae691ec5ad5652cf Bird Measurer (Assessment)         1   
001d0ed0        7b536271e99518f0 Bird Measurer (Assessment)         5   
00225f67        619b9c069cf790ca Bird Measurer (Assessment)         2   
002db7e3        5025f22d6e944533 Bird Measurer (Assessment)         5   
                8f50f2fd987cade4 Bird Measurer (Assessment)         2   
                cace4c493ac347e3 Bird Measurer (Assessment)        11   
003372b0        28113cd58d865372 Bird Measurer (Assessment)         2   
004c2091        343206b33bfa7941 Bird Measurer (Assessment)         1   
                4e446ee715d1bd94 Bird Measurer (Assessment)         1   
00691033        57e99afbe299eee7 Bird Measurer (Assessment)         7   
                9d2a83e6be787757 Bird Measurer (Assessment)         3   
                a4060f56e5b8b57b Bird Measurer (Assessment)         7   
                f76ce167ea7e64ac Bird Measurer (Assessment)         8   
00e17272        6c8fab7964639f37 Bird Measurer (Assessment)         2   
00e536bf        06e4408045163fdd Bird Measurer (Assessment)         3   
00fa8681        2eafd59d5ecf1b1e Bird Measurer (Assessment)         3   
00fc65b6        469b3ff25bf7824f Bird Measurer (Assessment)         1   
010bc1d5        49f65396b901366f Bird Measurer (Assessment)         2   
01120f12        f5fea737ce83acc6 Bird Measurer (Assessment)         2   
0153c957        65f9c6e01c6865d3 Bird Measurer (Assessment)         6   
01825124        39f3e52339d99ca2 Bird Measurer (Assessment)         1   
                f8de7f9f1baedba3 Bird Measurer (Assessment)         4   
01bdd720        2cbd37f1442b2b80 Bird Measurer (Assessment)         1   
01d5e1f5        bb3738392f5a4ae0 Bird Measurer (Assessment)         1   
01f71702        ccd4bb61d0b2efd3 Bird Measurer (Assessment)         1   
                d365ff77f064a4b3 Bird Measurer (Assessment)         2   
01faa4bf        09c766b97c48a6c8 Bird Measurer (Assessment)         1   
...                                                               ...   
fc20de6e        055c34e01290a4cc Bird Measurer (Assessment)         4   
                57ef618c34bef564 Bird Measurer (Assessment)         7   
fc2aa3b2        c3f8546446237a00 Bird Measurer (Assessment)        11   
fc30ec02        1ff9465a6473530c Bird Measurer (Assessment)         1   
                7a7e795c104912df Bird Measurer (Assessment)         1   
                9c4a86c4ed6325e9 Bird Measurer (Assessment)         2   
                bd739d4cb1d83693 Bird Measurer (Assessment)         6   
fc3eae77        e3f3cfc6f2dc72a7 Bird Measurer (Assessment)         5   
fc4ad96c        876b52f1d21ac365 Bird Measurer (Assessment)         9   
fc52a88f        1d27f3a09ad62ed6 Bird Measurer (Assessment)         5   
fcb0a469        ed1f838403677860 Bird Measurer (Assessment)        14   
fce2b016        f711e28b0e487c4a Bird Measurer (Assessment)         1   
fcecab06        23a02d67995f1b96 Bird Measurer (Assessment)         2   
fd02a7e0        c0b6ec9f5dfa7887 Bird Measurer (Assessment)         1   
fd1c8d0b        3fbfd10134b5add6 Bird Measurer (Assessment)         1   
fd5385c7        0024681d73e46aca Bird Measurer (Assessment)         1   
                479275730b2263fe Bird Measurer (Assessment)         1   
fdf4eb95        640c8948075d8838 Bird Measurer (Assessment)         1   
fe488283        bd6bea029a3bfb29 Bird Measurer (Assessment)         4   
                e2a93364fe4e41d2 Bird Measurer (Assessment)         4   
fe4a63a7        d8b934fef20a7a18 Bird Measurer (Assessment)         1   
fe4d880a        f03275b2cd195b86 Bird Measurer (Assessment)         2   
fe5f0699        e05c63355f859eb6 Bird Measurer

def make_labels(data):
    
    """
     1. Ne garder que les lignes où l'event_code vaut 4100 ou 4110: correspond aux event finaux d'une game_session
         donc contennant l'info de succés ou d'echec
         
     2. groupby par game_session
     
     3. Au sein de chaque game_session compter le nombre de succés et d'echecs
     
     4. Avec un merge créer un tableau contennant nombre de succés et nombre d'echecs pour chaque game_session. 
         /!\ Il peut y avoir des game_session pour lesquelles il n'y a que des succes ou que des echecs dans ce cas
         la case vide corespondante est NA que l'on remplace par 0.
         
     5. Calculer l'accuracy_score = nb_succes / nb_tentatives ou nb_tentatives = nb_succes + nb_echecs
     
     6. Construire la variable accuracy: repartition dans les classes (0,1,2,3)
     
     7. Ajouter au DataFrame final les variables de train informatives
    """
    
    # 1 
    data_assignement = data[(data['event_code'] == 4100) | (data['event_code'] == 4110)]
    
    # 2-3
    num_correct = pd.DataFrame(data_assignement[data_assignement['event_data'].str.contains('correct":true')].groupby('game_session').count()["event_id"].rename('num_correct'))
    
    num_incorrect = pd.DataFrame(data_assignement[data_assignement['event_data'].str.contains('correct":false')].groupby('game_session').count()["event_id"].rename('num_incorrect'))
    
    # 4
    data_l = pd.DataFrame(num_correct.merge(num_incorrect, how='outer', left_on=num_correct.index, right_on=num_incorrect.index)).fillna(0)
    data_l = pd.DataFrame(data_l.rename(columns={'key_0':'game_session', 'num_correct_x':'num_correct', 'num_correct_y':'num_incorrect'}))
    
    # 5
    data_l["accuracy"] = data_l["num_correct"]/(data_l["num_correct"]+data_l["num_incorrect"])
    
    # 6 
    data_l["accuracy_group"] = data_l["accuracy"].apply(lambda x: 0 if x==0 else (1 if x<0.5 else (2 if x<0.9 else 3)))
    
    data_l = data_l.merge(train[['installation_id', 'game_session', 'title','event_data','event_count','game_time','world','timestamp']], how='inner', left_on='game_session', right_on='game_session')

    return data_l